In [4]:
from langchain.llms import LlamaCpp
import pandas as pd
import swifter
from multiprocessing import Pool
import re
import os
def remove_html_tags(text):
    """Remove html tags from a string"""
    import re
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)


In [5]:
FILE_PATH="/Users/aiswaryaramachandran/Library/CloudStorage/OneDrive-Personal/bikewala/Harley_X440_reviews.csv"


In [6]:
data=pd.read_csv(FILE_PATH)
data.shape

(22, 38)

In [7]:
data.head()

,STARS,MAKE,MODEL,MODELID,ORIGINALIMGPATH,ORIGINALIMAGEPATH,HOSTURL,DESCRIPTION,SANITIZEDDESCRIPTION,SHORTDESCRIPTION,...,REVIEWAGE,ISRATINGQUESTION,RATINGQUESTIONSCOUNT,REVIEWURL,THANKSHEADING,THANKSTEXT,TOTALRATINGS,TOTALREVIEWS,OVERALLMODELRATING,REPORTEDMILEAGE
0,5,Harley-Davidson,X440,2453,NaN,NaN,NaN,The buying experience of the Harley Davidson x...,The buying experience of the Harley Davidson x...,The buying experience of the Harley Davidson x...,...,4 months ago,True,3,https://www.bikewale.com/harleydavidson-bikes/...,Tell us more about your experience,Write a detailed review and help fellow riders...,0,0,0.0,0
1,5,Harley-Davidson,X440,2453,NaN,NaN,NaN,This is my first Harley Davidson bike. I have ...,This is my first Harley Davidson bike. I have ...,This is my first Harley Davidson bike. I have ...,...,2 weeks ago,True,3,https://www.bikewale.com/harleydavidson-bikes/...,Tell us more about your experience,Write a detailed review and help fellow riders...,0,0,0.0,32
2,5,Harley-Davidson,X440,2453,NaN,NaN,NaN,Buying experience was good as it was from the ...,Buying experience was good as it was from the ...,Buying experience was good as it was from the ...,...,2 weeks ago,True,3,https://www.bikewale.com/harleydavidson-bikes/...,Tell us more about your experience,Write a detailed review and help fellow riders...,0,0,0.0,26
3,5,Harley-Davidson,X440,2453,NaN,NaN,NaN,Harley-Davidson X440 is a powerful motorcycle ...,Harley-Davidson X440 is a powerful motorcycle ...,Harley-Davidson X440 is a powerful motorcycle ...,...,4 weeks ago,True,2,https://www.bikewale.com/harleydavidson-bikes/...,Tell us more about your experience,Write a detailed review and help fellow riders...,0,0,0.0,35
4,5,Harley-Davidson,X440,2453,NaN,NaN,NaN,"Very convenient experience, staff were very he...","Very convenient experience, staff were very he...","Very convenient experience, staff were very he...",...,4 weeks ago,True,3,https://www.bikewale.com/harleydavidson-bikes/...,Tell us more about your experience,Write a detailed review and help fellow riders...,0,0,0.0,35


In [8]:
data['cleaned_description']=data['DESCRIPTION'].apply(lambda x:remove_html_tags(x))

In [9]:
text="\n".join(data['cleaned_description'])


In [10]:
print(text)

The buying experience of the Harley Davidson x440 is a seamless and professional process. From exploring its website to visiting a dealership, the brand ensures that customers are well-informed about the product's features, specifications, and pricing. The sales representatives are knowledgeable and attentive, answering all queries promptly and guiding buyers in selecting the most suitable model. Once purchased, the riding experience on the Harley Davidson x440 is unparalleled. With its powerful engine and superior handling, it effortlessly maneuvers through various terrains with precision and stability. Its sleek design exudes sophistication while maintaining its iconic Harley Davidson identity. The attention to detail in every aspect of this motorcycle is evident from its exceptional build quality to its comfortable seating position. When it comes to performance, the x440 delivers without compromise; whether cruising on highways or tackling challenging corners, it offers unmatched po

In [14]:
text

"The buying experience of the Harley Davidson x440 is a seamless and professional process. From exploring its website to visiting a dealership, the brand ensures that customers are well-informed about the product's features, specifications, and pricing. The sales representatives are knowledgeable and attentive, answering all queries promptly and guiding buyers in selecting the most suitable model. Once purchased, the riding experience on the Harley Davidson x440 is unparalleled. With its powerful engine and superior handling, it effortlessly maneuvers through various terrains with precision and stability. Its sleek design exudes sophistication while maintaining its iconic Harley Davidson identity. The attention to detail in every aspect of this motorcycle is evident from its exceptional build quality to its comfortable seating position. When it comes to performance, the x440 delivers without compromise; whether cruising on highways or tackling challenging corners, it offers unmatched p

In [28]:
llm = LlamaCpp(
    # You can pass in the URL to a GGML model to download it automatically
    #model_url="https://huggingface.co/TheBloke/Llama-2-13B-chat-GGML/resolve/main/llama-2-13b-chat.ggmlv3.q4_0.bin",
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    #model_path="/Users/aiswaryaramachandran/.cache/lm-studio/models/Aiswarya/LLAMA-2-7B-CHAT-GGUF/llama-2-7b-chat.gguf.q4_0.bin",
    model_path="/Users/aiswaryaramachandran/.cache/lm-studio/models/TheBloke/zephyr-7B-beta-GGUF/zephyr-7b-beta.Q8_0.gguf",
    n_gpu_layers=1,
    n_batch=512,
    temperature=0.01,
    max_tokens=1000,
    top_p=0.95,
    verbose=True, # Verbose is required 
    f16_kv=True,
    
    n_ctx=32768,stop=["###","Input:"],
    use_mlock=True,
    repeat_penalty=1.1,
    top_k=40

)

llama_model_loader: loaded meta data with 21 key-value pairs and 291 tensors from /Users/aiswaryaramachandran/.cache/lm-studio/models/TheBloke/zephyr-7B-beta-GGUF/zephyr-7b-beta.Q8_0.gguf (version unknown)
llama_model_loader: - tensor    0:                token_embd.weight q8_0     [  4096, 32000,     1,     1 ]
llama_model_loader: - tensor    1:           blk.0.attn_norm.weight f32      [  4096,     1,     1,     1 ]
llama_model_loader: - tensor    2:            blk.0.ffn_down.weight q8_0     [ 14336,  4096,     1,     1 ]
llama_model_loader: - tensor    3:            blk.0.ffn_gate.weight q8_0     [  4096, 14336,     1,     1 ]
llama_model_loader: - tensor    4:              blk.0.ffn_up.weight q8_0     [  4096, 14336,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_norm.weight f32      [  4096,     1,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.attn_k.weight q8_0     [  4096,  1024,     1,     1 ]
llama_model_loader: - tensor    7:    

In [33]:
prompt="Answer the Question based on the text  :  \n"+text+"\n Based on the text material above, generate the response to the following quesion or instruction: What are some of the cons of the Harley Davidson X440, according to the reviews and experiences shared by users?"

In [34]:
llm(prompt)

Llama.generate: prefix-match hit

llama_print_timings:        load time =  1840.12 ms
llama_print_timings:      sample time =     0.74 ms /     1 runs   (    0.74 ms per token,  1351.35 tokens per second)
llama_print_timings: prompt eval time = 11106.30 ms /  2630 tokens (    4.22 ms per token,   236.80 tokens per second)
llama_print_timings:        eval time =     0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings:       total time = 11117.37 ms


''

In [35]:

'''
<|system|>
Below is an instruction that describes a task. Write a response that appropriately completes the request.

<|user|>
Answer all the questions in this Chat, based on the text below :  "The buying experience of the Harley Davidson x440 is a seamless and professional process. From exploring its website to visiting a dealership, the brand ensures that customers are well-informed about the product's features, specifications, and pricing. The sales representatives are knowledgeable and attentive, answering all queries promptly and guiding buyers in selecting the most suitable model. Once purchased, the riding experience on the Harley Davidson x440 is unparalleled. With its powerful engine and superior handling, it effortlessly maneuvers through various terrains with precision and stability. Its sleek design exudes sophistication while maintaining its iconic Harley Davidson identity. The attention to detail in every aspect of this motorcycle is evident from its exceptional build quality to its comfortable seating position. When it comes to performance, the x440 delivers without compromise; whether cruising on highways or tackling challenging corners, it offers unmatched power and responsiveness. In conclusion, owning a Harley Davidson x440 truly embodies the spirit of biking excellence coupled with an exceptional buying experience.
This is my first Harley Davidson bike. I have been waiting for the last 2 years for a budget-friendly and luxury-looking cruiser bike. This fits my expectations very well. Service should be done after 10000 km or every one year. Always take a service from the agency for the betterment of the bike. Overall performance is awesome.
Buying experience was good as it was from the Harley showroom, in Hyderabad. I booked for the top end but as it was taking more time to get delivery took the vivid model, which is the value-for-money model. 
The riding experience is superb, it's already been 1 month and 500km I am using it but still searching for opportunities to ride my bike again and again. The power and torque are just wow. I ride mostly in the Hyderabad city traffic but my experience is very good. Easy to handle even in traffic. And that quick maneuver when the signal opens is superb.
The looks and build quality is very good. One thing is the wire/cable management would have been better. The front riding handle cable as well as the cable that went to the rear wheel brake mechanism could have been better quality. The mirror is not sufficient for the rearview. 
Handling is very good and the break performance is very good.
Not yet done the first service.
Compared to RE, which will cost you 3L onroad, it gives the same 15year engine with cosmetic changes with different color options we can go for X440 with some 50K additional for the improved and powerful engine 440cc, 27bhp with 38NM torque. All LED light setup with beautiful looks and signature DRL and projector lamp.
Harley-Davidson X440 is a powerful motorcycle with a comfortable seating position and a refined, ergonomic design. Performance The X440 is powerful and responsive. Handling The X440 is easy to handle and stable at high speed. The X440 has an assist and slipper clutch. There are a few things for make the x440 the best bike.
Very convenient experience, staff were very helpful. Great and  Comfortable, torque and power are very, recommended as compared to torque and handling. Slim decent look, charming personality, and performance are also very impressive with their looks. Compatible friendly services at the service station, maintaining is cheaper as you get used to it. It gives a very good personality, speed, power, and handling is awesome, compatible mileage. I think it is a kind of lightweight as a brand company it should be much bulkier.
It's a very good bike for riding. It's also good for a long journey. When it comes to performance, the x440 delivers without compromise; whether cruising on highways or tackling challenging corners, it offers unmatched power and responsiveness. In conclusion, owning a Harley Davidson x440 truly embodies the spirit of biking excellence coupled with an exceptional buying experience.
I bought the Harley Davidson X440 S variant. I got this delivered on 26th October. And have driven in Bangalore traffic conditions as well as taken it out.
Buying Experience:
The buying experience was good and the Harley staff behaviour was very nice.
Riding Experience :
This is the most unexpected part. I have driven a splendour before this for 6 years and was afraid that in traffic, I would have to struggle. But I was blown away by the handling of this motorcycle. Once u have switched it on, you don't feel the weight. The motorcycle was able to easily come out of traffic.
The response of the bike is amazing, and as soon as you pull the accelerator, you feel the power. And for one long trip I did, this bike feels amazing on the 5th and 6th gear. A relaxed effortless cruising experience. 
Pros :
1. Great Handing
2. Smooth Cruising Experience
3. Great throttle response
4. Eye-catching, absolutely stunning-looking bike
5. Road Presence 
6. Great suspension
Dream bike for me, have no other experience with this bike but its look and performance look quite awesome with modern technology.
It weighs less than the Royal Enfield classic and its performance is better than RE.
The sound is lovable as heard in the video review just eagerly waiting for that day when I own this real road king.
Looks awesome like a typical HarleySound feels great and the seating position looks promising. I am going for this bike. It's definitely the best among the rivals in all aspects. 
Though the engine is made by Hero, I don't see any hero are much more durable and less maintained. 
What else do we need.. its a perfect motorcycle to go for all age groups.
The pricing of the base variant is good. But lack in features like navigation,gear indicators,traction control,only air cooling system.  But the ergonomics seems good . The ride will be comfortable. Harley provides lcd display but no navigation no connectivity features etc. But harley didn't overpriced it. I think it's a value for money. At least it is a Harley Davidson . In the books they claim that this bike gets 35 km mileage,that is a decent mileage
Firstly the bike was excellent good milage the engine capacity was soo good in this budget soo good experience in riding this bike good performance excellent milage use tube less tyres for vloggers and riders for tours after the service bike was so fresh like a showroom bike it was soo good bike I have ever seen
Amazing bike from harley and hero collaboration.With both front and rear disc brakes, Harley-Davidson X440 comes up with anti-locking braking system. This X440 bike weighs 190.5 kg and has a fuel tank capacity of 13.5 liters.2.29 Lakh and goes upto Rs. 2.69 Lakh. Harley Davidson X440 is offered in 3 variants - Denim, Vivid and the top variant X440 S which comes at a price tag of Rs. 2.69 Lakh.
Great experience i recently purchased this bike and my riding experience is very out standing wow wow wow harley davidson it's sports bike ilove this bike looks premium and sharap performance is okay not a great but one thing is milega is very low and service and maintenance charges is higher than other normal bikes but it's not major things for sports bike lovers and iam fully satisfied with this bike and recommend this bike for purchase all people from india great experience harley davidson top speed is 130❤️❤️🥰🥰
Waited for 3 month to get this product launch and now even after launch delivery dates are of october.
This is really bad marketing. Why to launch a bike in so much advance when it's not available to deliver and also there is no bike available for display or test ride. Who wants to book a bike worth 3 lakh without sitting on it taking a nearby look.
Finally i bought honda h'ness cb350 and happy with my purchase .
Fully comfortable to ride for 400-500 Km and the bike mileage is normal the bike Bike is very good and the price is satisfactory. I really loved this Harley Davidson x440.  its competitors especially better than Royal Enfield Classic 350, Triumph Speed 400, Royal Enfield Hunter 350, etc. I will definitely buy this bike for sure.
Harley is a dream for all indians and in this price range wah from all indians, this bike will be one of the best segments in the indian bikes category, look and rate are almost unique. At the outset, i have to admit that i admire harley-davidson for their new approach to motorcycles. For a manufacturer that’s steeped in history with big, air-cooled v-twin powered bikes dominating their portfolio, and to some extent their identity, the new line of harley-davidson motorcycles represent their efforts of moving with the times.
I don't buy this bike.... But I will but it definitely if my budget is upto 2.5 lakh. It is best among all of its competitors especially better then Royal Enfield
Harley-davidson has a better brand value then Royal Enfield it's like to buy a I phone instead of samsung. One day I will definitely buy this bike for sure
This bike is very first bike introduced by Harley Davidson in India. I hope this bike will not disappoint us. Another reason to love this bike is that this bike is gonna be super cool and the looks of this bike is very awesome 👍  at last I just want to say that I really loved this Harley davidson x440
Bike is very good and the price is satisfactory the bike mileage is normal the bike suspension is gorgeous I love this bike's sound you will buy this bike the tyre and suspension is good  and the bike engine  is very powerful engine the gears 5 gears it's very good bike in this price
Great to bring this model to my country. people will accept this motor cycle. Really it will going to grab the India roads .bike looks good.hero and harley David son both companies will hit the market by giving this motor cycle and it's dream to buy this motor cycle for best price and I'm going to buy this as soon as possible and it's my dream
Pros:
Riding experience was awesome. The comfort is great. The exhaust note is just WOW! This bike is a blend of modern as well as retro. Amazing features like auto glow headlamps and Harley badging on the headlamp. 
Cons:
The gap between the fuel tank and the headlamp area seems a bit empty from the side profile. 
Suspensions could have been much better for bumpy rides.
The buying experience was fantastic. I got this on the first day of x440 delivery that too first bike owned by SR Nagar Bhanu Automobiles. There were regional managers present from Harley-Davidson, it was a great feeling.
The bike was amazing in its looks the sound was mind-blowing and the pickup was beyond my expectations."
<|assistant|>
Based on the customer testimonials and product description provided, here's my response:

Overall, customers who have purchased the Harley Davidson x440 are highly satisfied with their buying experience and riding performance. The seamless and professional process of exploring the brand's website and visiting a dealership has left them well-informed about the product's features, specifications, and pricing. The sales representatives at the dealership have been knowledgeable, attentive, and prompt in answering queries, guiding buyers through the selection process, and providing exceptional service.

The riding experience on the Harley Davidson x440 has been described as unparalleled, with a powerful engine, superior handling, and sleek design that exudes sophistication while maintaining its iconic Harley Davidson identity. The attention to detail in every aspect of this motorcycle is evident from its exceptional build quality, comfortable seating position, and outstanding performance on both highways and challenging corners.

Customers have noted that the Harley Davidson x440 delivers without compromise and provides a relaxed, effortless cruising experience. They appreciate the bike's handling, which allows it to easily maneuver through traffic, and its smooth throttle response, exceptional suspension, and road presence. Some customers also mentioned that the Harley Davidson x440 weighs less than its competitors and has better performance.

Customers have praised the bike's modern technology, including auto glow headlamps and Harley badging on the headlamp, as well as its blend of modern and retro features. They also highlighted the sound of the exhaust note, which they found to be absolutely WOW! Some customers mentioned that there is a gap between the fuel tank and the headlamp area, but overall, the bike's comfort and style were highly praised.

In terms of maintenance, some customers suggested that wire/cable management could have been better, but overall, they found the riding experience and service to be exceptional. They also noted that the Harley Davidson x440 was a good value for money compared to its competitors, with comparable engine specifications and superior performance.

In conclusion, the buying and riding experiences of the Harley Davidson x440 have been overwhelmingly positive, with customers praising the brand's professionalism, the bike's exceptional performance, handling, comfort, style, and value for money. The Harley Davidson x440 truly embodies the spirit of biking excellence coupled with an exceptional buying experience.
<|user|>
What has been the experience related to mileage of Harley X440?
<|assistant|>
The provided text does not offer a consistent or comprehensive insight into the experience related to mileage of the Harley Davidson x440. While some customers have mentioned their positive experiences with the bike's performance and cruising experience on highways, they did not specifically discuss the mileage of the motorcycle. Therefore, it is unclear what the average or estimated mileage of the Harley Davidson x440 is. It is best to consult the official specifications or contact an authorized dealer for more information regarding the bike's mileage.
<|user|>
What are the main cons of Harley X440?
<|assistant|>
Based on the provided responses, there seem to be very few cons mentioned about the Harley Davidson X440. Here are a few potential areas that some users have noted as less favorable:

1. Wire/cable management could be improved: Some users have commented that the wire and cable quality for the front riding handle and rear wheel brake mechanism could be better.

2. Mirror could be larger: A few individuals have mentioned that the mirror on the X440 could be larger to provide a better view of what's behind the bike.

3. Price may be a factor: While some users believe that the price point is justified due to the bike's performance and features, others have noted that it may be too expensive for their budget.

4. Service intervals are shorter: Harley recommends getting an X440 service every 10,000 kilometers or once a year, which some buyers find more frequent than they would prefer.

Overall, however, the feedback from users has been overwhelmingly positive about the Harley Davidson X440's performance, handling, and overall experience.
<|user|>
What are the key points mentioned regarding the performance of Harley X440?
<|assistant|>
Based on the provided responses, here are some key points related to the performance of the Harley Davidson X440:

1. Powerful engine: Many users have praised the X440's 443cc V-twin engine for its power and torque, describing it as "wow" and "amazing."

2. Responsive acceleration: The bike's throttle response is noted as exceptional, with users describing it as "great" and "instantaneous."

3. Smooth cruising experience: The X440's handling and smoothness while cruising on highways are frequently mentioned by buyers, who describe it as "unmatched" and "effortless."

4. Excellent for tackling corners: Several users have noted the X440's ability to handle challenging corners with precision and stability.

5. Assist and slipper clutch: The bike features an assist and slipper clutch, which makes shifting gears easier and smoother, particularly when downshifting.

Overall, buyers seem very satisfied with the X440's performance, describing it as "exceptional," "amazing," and "unmatched."
<|user|>
What are the pain points mentioned regarding Suspension in Harley X440?
<|assistant|>
Based on the provided responses, there do not appear to be any specific pain points related to suspension in the Harley Davidson X440. In fact, some users have even praised the bike's suspension for its great performance. One user mentioned that the suspension is "good" and another described it as "great." No issues or concerns regarding suspension were raised by other buyers either. So, we can say that the suspension of Harley X440 seems to be satisfactory based on the user feedback provided.

'''

'\n<|system|>\nBelow is an instruction that describes a task. Write a response that appropriately completes the request.\n\n<|user|>\nAnswer all the questions in this Chat, based on the text below :  "The buying experience of the Harley Davidson x440 is a seamless and professional process. From exploring its website to visiting a dealership, the brand ensures that customers are well-informed about the product\'s features, specifications, and pricing. The sales representatives are knowledgeable and attentive, answering all queries promptly and guiding buyers in selecting the most suitable model. Once purchased, the riding experience on the Harley Davidson x440 is unparalleled. With its powerful engine and superior handling, it effortlessly maneuvers through various terrains with precision and stability. Its sleek design exudes sophistication while maintaining its iconic Harley Davidson identity. The attention to detail in every aspect of this motorcycle is evident from its exceptional b

In [46]:
prompt='''<|system|>
Below is an instruction that describes a task. Write a response that appropriately completes the request.

<|user|>
Answer all the questions in this Chat, based on the text below : '''+text+"<|assistant|>"

In [47]:
output=llm(prompt)

Llama.generate: prefix-match hit

llama_print_timings:        load time =  1840.12 ms
llama_print_timings:      sample time =   113.56 ms /   155 runs   (    0.73 ms per token,  1364.98 tokens per second)
llama_print_timings: prompt eval time = 10526.59 ms /  2596 tokens (    4.05 ms per token,   246.61 tokens per second)
llama_print_timings:        eval time =  5952.03 ms /   154 runs   (   38.65 ms per token,    25.87 tokens per second)
llama_print_timings:       total time = 16830.86 ms


In [48]:
print(output)


Overall, the Harley Davidson x440 is an exceptional motorcycle that delivers on performance, style, and value for money. From the buying experience to the riding experience, customers are impressed with the brand's attention to detail and commitment to customer satisfaction. The bike's powerful engine, superior handling, and sleek design make it a standout in its segment, and its competitive pricing makes it an attractive option for biking enthusiasts. Whether cruising on highways or tackling challenging corners, the Harley Davidson x440 offers unmatched power and responsiveness that embodies the spirit of biking excellence. It's no wonder that this bike has received rave reviews from customers and industry experts alike.


In [49]:
prompt=prompt+output
prompt

"<|system|>\nBelow is an instruction that describes a task. Write a response that appropriately completes the request.\n\n<|user|>\nAnswer all the questions in this Chat, based on the text below : The buying experience of the Harley Davidson x440 is a seamless and professional process. From exploring its website to visiting a dealership, the brand ensures that customers are well-informed about the product's features, specifications, and pricing. The sales representatives are knowledgeable and attentive, answering all queries promptly and guiding buyers in selecting the most suitable model. Once purchased, the riding experience on the Harley Davidson x440 is unparalleled. With its powerful engine and superior handling, it effortlessly maneuvers through various terrains with precision and stability. Its sleek design exudes sophistication while maintaining its iconic Harley Davidson identity. The attention to detail in every aspect of this motorcycle is evident from its exceptional build 

In [50]:
prompt=prompt+"|<user>|What are the main cons of Harley X440?|<assistant>|"

In [51]:
output=llm(prompt)

Llama.generate: prefix-match hit

llama_print_timings:        load time =  1840.12 ms
llama_print_timings:      sample time =   269.89 ms /   305 runs   (    0.88 ms per token,  1130.07 tokens per second)
llama_print_timings: prompt eval time =   526.79 ms /    24 tokens (   21.95 ms per token,    45.56 tokens per second)
llama_print_timings:        eval time = 12134.75 ms /   304 runs   (   39.92 ms per token,    25.05 tokens per second)
llama_print_timings:       total time = 14057.97 ms


In [52]:
output

"Based on the feedback provided by some customers, here are a few cons of the Harley Davidson x440:\n1. Wire/cable management could be better, particularly for the front riding handle cable and the cable that goes to the rear wheel brake mechanism.\n2. The mirror provided is not sufficient for rearview visibility.\n3. The gap between the fuel tank and the headlamp area seems a bit empty from the side profile.\n4. Suspensions could have been better for bumpy rides.\n5. Some customers feel that the bike's mileage is on the lower side, and service and maintenance charges are higher than other normal bikes in its segment.\n6. Delayed delivery dates after launch, which some customers find disappointing as they prefer to test drive and sit on the bike before making a purchase decision.\n7. The price of the base variant is good, but it lacks features like navigation, gear indicators, traction control, and only has air cooling system.\n8. Some customers feel that the bike's weight could be lig

In [53]:
print(output)

Based on the feedback provided by some customers, here are a few cons of the Harley Davidson x440:
1. Wire/cable management could be better, particularly for the front riding handle cable and the cable that goes to the rear wheel brake mechanism.
2. The mirror provided is not sufficient for rearview visibility.
3. The gap between the fuel tank and the headlamp area seems a bit empty from the side profile.
4. Suspensions could have been better for bumpy rides.
5. Some customers feel that the bike's mileage is on the lower side, and service and maintenance charges are higher than other normal bikes in its segment.
6. Delayed delivery dates after launch, which some customers find disappointing as they prefer to test drive and sit on the bike before making a purchase decision.
7. The price of the base variant is good, but it lacks features like navigation, gear indicators, traction control, and only has air cooling system.
8. Some customers feel that the bike's weight could be lighter, par

In [58]:
prompt=prompt+output+"|<user>| What are the key points mentioned regarding the performance of Harley X440? \n |<assistant>|"

In [59]:
output=llm(prompt)

Llama.generate: prefix-match hit

llama_print_timings:        load time =  1840.12 ms
llama_print_timings:      sample time =   256.31 ms /   298 runs   (    0.86 ms per token,  1162.65 tokens per second)
llama_print_timings: prompt eval time =   582.46 ms /    30 tokens (   19.42 ms per token,    51.51 tokens per second)
llama_print_timings:        eval time = 12699.54 ms /   297 runs   (   42.76 ms per token,    23.39 tokens per second)
llama_print_timings:       total time = 14616.58 ms


In [60]:
print(output)

 Based on the feedback provided by some customers, here are the key points mentioned regarding the performance of the Harley Davidson x440:
1. The bike's engine is powerful and responsive, with a 443cc Revolution X-V Twin engine that delivers 27bhp at 8250rpm and 38Nm of torque at 6000rpm.
2. Handling is easy and stable, even in traffic conditions.
3. The bike's quick maneuver when the signal opens is superb.
4. The bike's assist and slipper clutch make it convenient to ride.
5. The bike's throttle response is amazing, and once you switch it on, you don't feel the weight.
6. The bike's suspension is comfortable for long journeys, particularly in the 5th and 6th gear.
7. The bike's sound is lovable, as heard in video reviews.
8. The bike's performance is better than its competitors like the Royal Enfield Classic and Triumph Speed 400.
9. The bike's mileage is normal, but some customers feel that it could be higher.
10. The bike's top speed is 130km/h, which may not appeal to all sports 

In [61]:
prompt=prompt+output+"|<user>| What are the users comments about mileage of Harley X440 \n |<assistant>|"

In [62]:
output=llm(prompt)

Llama.generate: prefix-match hit

llama_print_timings:        load time =  1840.12 ms
llama_print_timings:      sample time =   106.90 ms /   133 runs   (    0.80 ms per token,  1244.15 tokens per second)
llama_print_timings: prompt eval time =   631.70 ms /    28 tokens (   22.56 ms per token,    44.32 tokens per second)
llama_print_timings:        eval time =  5793.27 ms /   132 runs   (   43.89 ms per token,    22.79 tokens per second)
llama_print_timings:       total time =  6860.12 ms


In [63]:
print(output)

 Based on the feedback provided by some customers, here are the users' comments about the mileage of the Harley Davidson x440:
1. Some customers feel that the bike's mileage is normal and could be higher.
2. The bike's mileage is mentioned as 35kmpl in the brochure, but some customers find this claim to be exaggerated.
3. However, other customers are satisfied with the bike's mileage and do not consider it a major concern.
4. Overall, the bike's performance and features outweigh concerns about mileage for many customers.


In [64]:
prompt=prompt+output+"|<user>| Which bikes are compared with Harley X440? \n |<assistant>|"

In [65]:
output=llm(prompt)
print(output)

Llama.generate: prefix-match hit


 Based on the feedback provided by some customers, here are the bikes that are compared with the Harley Davidson x440:
1. Royal Enfield Classic
2. Triumph Speed 400
3. Royal Enfield Hunter 350



llama_print_timings:        load time =  1840.12 ms
llama_print_timings:      sample time =    49.75 ms /    57 runs   (    0.87 ms per token,  1145.75 tokens per second)
llama_print_timings: prompt eval time =   579.30 ms /    26 tokens (   22.28 ms per token,    44.88 tokens per second)
llama_print_timings:        eval time =  2485.95 ms /    56 runs   (   44.39 ms per token,    22.53 tokens per second)
llama_print_timings:       total time =  3341.63 ms


In [18]:
sys_prompt='''<|system|>
Below is an instruction that describes a task. Write a response that appropriately completes the request.
Answer all the questions in this Chat, based on the text below : '''+text+"\n<|user|>"

In [19]:
prompt=sys_prompt+"What are the users comments about mileage of Harley X440 \n |<assistant>|"

In [20]:
output=llm(prompt)

Llama.generate: prefix-match hit

llama_print_timings:        load time =  1636.73 ms
llama_print_timings:      sample time =   103.84 ms /   126 runs   (    0.82 ms per token,  1213.45 tokens per second)
llama_print_timings: prompt eval time =     0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =  5048.22 ms /   126 runs   (   40.07 ms per token,    24.96 tokens per second)
llama_print_timings:       total time =  5492.09 ms


In [21]:
print(output)

>
Some users have mentioned that the mileage of Harley Davidson X440 is normal, while others have expressed that it's not great but satisfactory. Specific mileage figures mentioned by users range from 35 kmpl (claimed by the manufacturer) to around 25-30 kmpl in real-world conditions. However, it should be noted that mileage can vary based on factors such as riding style, terrain, and maintenance. Overall, mileage seems to be a minor concern for users who are more focused on the bike's performance, looks, and brand value.


In [29]:
prompt=sys_prompt+"What are the  cons of Harley X440?\n|<assistant>|"

In [30]:
output=llm(prompt)
print(output)

>
Based on the responses provided, some cons mentioned by users include:

1. Wire/cable management could be better for the front riding handle cable and rear wheel brake mechanism.
2. The mirror could be insufficient for the rearview.
3. The gap between the fuel tank and headlamp area seems a bit empty from the side profile.
4. Suspensions could have been much better for bumpy rides.
5. Some users mentioned that the mileage is low compared to other bikes in its segment.
6. Delayed delivery times, as some users had to wait several months after booking their bike.
7. The price of the base variant is good, but it lacks features like navigation, gear indicators, and traction control, which are available in competitors' bikes.
8. Some users found the service and maintenance charges to be higher than other normal bikes.



llama_print_timings:        load time =  1647.85 ms
llama_print_timings:      sample time =   142.80 ms /   196 runs   (    0.73 ms per token,  1372.53 tokens per second)
llama_print_timings: prompt eval time =  7955.75 ms /  2645 tokens (    3.01 ms per token,   332.46 tokens per second)
llama_print_timings:        eval time =  7519.02 ms /   195 runs   (   38.56 ms per token,    25.93 tokens per second)
llama_print_timings:       total time = 15913.15 ms


In [31]:
prompt=sys_prompt+"Can you pick lines from the review which talk about gap betwen fuel tank and headlamp area\n|<assistant>|"

In [32]:
output=llm(prompt)
print(output)

Llama.generate: prefix-match hit


Cons:
- The gap between the fuel tank and the headlamp area seems a bit empty from the side profile.



llama_print_timings:        load time =  1647.85 ms
llama_print_timings:      sample time =    19.15 ms /    26 runs   (    0.74 ms per token,  1357.42 tokens per second)
llama_print_timings: prompt eval time =   386.63 ms /    27 tokens (   14.32 ms per token,    69.83 tokens per second)
llama_print_timings:        eval time =   957.62 ms /    25 runs   (   38.30 ms per token,    26.11 tokens per second)
llama_print_timings:       total time =  1418.36 ms


In [37]:
prompt=sys_prompt+"Can you pick lines from the review which talk about cable or wire management\n|<assistant>|"

In [38]:
output=llm(prompt)
print(output)

Llama.generate: prefix-match hit



Yes, here are some lines from the review that discuss cable or wire management:

- "The front riding handle cable as well as the cable that went to the rear wheel brake mechanism could have been better quality."

- "Handling is very good and the break performance is very good. Not yet done the first service. Compared to RE, which will cost you 3L onroad, it gives the same 15year engine with cosmetic changes we can go for X440 with some 50K additional for the improved and powerful engine 440cc, 27bhp with 38NM torque. All LED light setup with beautiful looks and signature DRL and projector lamp."

- "The wire/cable management would have been better."



llama_print_timings:        load time =  1647.85 ms
llama_print_timings:      sample time =   147.44 ms /   170 runs   (    0.87 ms per token,  1153.03 tokens per second)
llama_print_timings: prompt eval time =   334.95 ms /    10 tokens (   33.49 ms per token,    29.86 tokens per second)
llama_print_timings:        eval time =  6526.22 ms /   169 runs   (   38.62 ms per token,    25.90 tokens per second)
llama_print_timings:       total time =  7603.20 ms


In [36]:
print(text)

The buying experience of the Harley Davidson x440 is a seamless and professional process. From exploring its website to visiting a dealership, the brand ensures that customers are well-informed about the product's features, specifications, and pricing. The sales representatives are knowledgeable and attentive, answering all queries promptly and guiding buyers in selecting the most suitable model. Once purchased, the riding experience on the Harley Davidson x440 is unparalleled. With its powerful engine and superior handling, it effortlessly maneuvers through various terrains with precision and stability. Its sleek design exudes sophistication while maintaining its iconic Harley Davidson identity. The attention to detail in every aspect of this motorcycle is evident from its exceptional build quality to its comfortable seating position. When it comes to performance, the x440 delivers without compromise; whether cruising on highways or tackling challenging corners, it offers unmatched po

In [39]:
prompt=sys_prompt+"Can you pick lines from the review which compare Harley X440 to Royal Enfield?\n|<assistant>|"

In [40]:
output=llm(prompt)
print(output)

Llama.generate: prefix-match hit


>
1. "Compared to RE, which will cost you 3L onroad, it gives the same 15year engine with cosmetic changes with different color options we can go for X440 with some 50K additional for the improved and powerful engine 440cc, 27bhp with 38NM torque. All LED light setup with beautiful looks and signature DRL and projector lamp."
2. "This bike is very first bike introduced by Harley Davidson in India. I hope this bike will not disappoint us. Another reason to love this bike is that this bike is gonna be super cool and the looks of this bike is very awesome 👍 at last I just want to say that I really loved this Harley davidson x440, better than Royal Enfield."
3. "The buying experience was fantastic. I got this on the first day of x440 delivery that too first bike owned by SR Nagar Bhanu Automobiles. There were regional managers present from Harley-Davidson, it was a great feeling. The bike was amazing in its looks the sound was mind-blowing and the pickup was beyond my expectations. Compare


llama_print_timings:        load time =  1647.85 ms
llama_print_timings:      sample time =   873.47 ms /  1000 runs   (    0.87 ms per token,  1144.86 tokens per second)
llama_print_timings: prompt eval time =   366.40 ms /    19 tokens (   19.28 ms per token,    51.86 tokens per second)
llama_print_timings:        eval time = 40811.97 ms /   999 runs   (   40.85 ms per token,    24.48 tokens per second)
llama_print_timings:       total time = 45814.25 ms
